<a href="https://colab.research.google.com/github/Melynna/Aula_01/blob/main/ProjetoP2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import hashlib
import os
import datetime
import pickle
from tabulate import tabulate

# Paths for credentials file
CREDENTIALS_FILE = "credentials.txt"
INVENTORY_FILE = "inventory.txt"

# In-memory data store for objects
inventory = {}

class MLB:
  def __init__(self, serial_number, username):
    self.serial_number = serial_number
    self.status = "checked_in"
    self.checked_in_at = None
    self.last_checked_in_by = None
    self.checked_out_at = None
    self.last_checked_out_by = None

  def check_in(self, username):
    self.checked_in_at = datetime.datetime.now()
    self.status = "checked_in"
    self.last_checked_in_by = username

  def check_out(self, username):
    self.checked_out_at = datetime.datetime.now()
    self.status = "checked_out"
    self.last_checked_out_by = username

def load_credentials():
    if not os.path.exists(CREDENTIALS_FILE):
        open(CREDENTIALS_FILE, 'w').close()  # Create file if not exists
    credentials = {}
    with open(CREDENTIALS_FILE, 'r') as file:
        for line in file:
            username, password_hash = line.strip().split(':')
            credentials[username] = password_hash
    return credentials

def save_credentials(username, password_hash):
    with open(CREDENTIALS_FILE, 'a') as file:
        file.write(f"{username}:{password_hash}\n")

def hash_password(password):
    return hashlib.sha256(password.encode()).hexdigest()

def authenticate_user(credentials):
    username = input("Enter username: ").strip()
    password = input("Enter password: ").strip()
    password_hash = hash_password(password)

    if username in credentials and credentials[username] == password_hash:
        print("Login successful!")
        return username
    else:
        print("Invalid username or password.")
        return None

def register_user(credentials):
    username = input("Enter new username: ").strip()
    if username in credentials:
        print("Username already exists.")
        return

    password = input("Enter new password: ").strip()
    password_hash = hash_password(password)
    save_credentials(username, password_hash)
    print("User registered successfully!")

def load_inventory():
    global inventory
    if not os.path.exists(INVENTORY_FILE) or os.path.getsize(INVENTORY_FILE) == 0:
        open(INVENTORY_FILE, 'w').close()
        return

    with open(INVENTORY_FILE, "rb") as file:  # Use 'rb' to read in binary mode
        inventory = pickle.load(file)

def save_inventory():
    with open(INVENTORY_FILE, "wb") as file:
        pickle.dump(inventory, file)

def check_in(username):
    mlb_serial_number = input("Enter the MLB Serial number: ").strip()
    input_bin = input("Enter the MLB Bin: ").strip()
    input_slot = input("Enter the MLB Bin Slot: ").strip()

    if not input_bin in inventory:
        inventory[input_bin] = {}

    if not input_slot in inventory[input_bin]:
        inventory[input_bin][input_slot] = MLB(mlb_serial_number, username)

    mlb = inventory[input_bin][input_slot]
    mlb.check_in(username)
    save_inventory()

    print(f"Object '{mlb_serial_number}' checked in by {username}.")

def check_out(username):
    input_bin = input("Enter the Bin number: ").strip()

    if not input_bin in inventory:
        print("Bin is empty.")
        return

    input_slot = input("Enter the Slot number: ").strip()

    if not input_slot in inventory[input_bin]:
        print("Slot is empty.")
        return

    mlb = inventory[input_bin][input_slot]
    mlb.check_out(username)
    save_inventory()

    print(f"MLB Succcessfully checked out by {username}.")


def show_inventory():
    headers = ["Bin", "Slot", "Serial Number", "Status", "Last Check In Date", "Last Check In By", "Last Check Out Date", "Last Check Out By"]
    data = []

    for mlb_bin, mlb_slots in inventory.items():
        for mlb_slot, mlb in mlb_slots.items():
            data.append([mlb_bin, mlb_slot, mlb.serial_number, mlb.status, mlb.checked_in_at, mlb.last_checked_in_by, mlb.checked_out_at, mlb.last_checked_out_by])

    print(tabulate(data, headers=headers, tablefmt="pretty"))

def main():
    credentials = load_credentials()
    load_inventory()

    print("Welcome to the Inventory Management System")
    print("1. Login")
    print("2. Register")

    username = None
    choice = input("Enter your choice: ").strip()
    print("\n")

    if choice == "1":
        username = authenticate_user(credentials)
    elif choice == "2":
        register_user(credentials)
        username = authenticate_user(credentials)
    else:
        print("Invalid input. Exiting.")
        return

    if username:
        while True:
            print("\nOptions:")
            print("1. Check in an object")
            print("2. Check out an object")
            print("3. View inventory")
            print("4. Exit")

            choice = input("Enter your choice: ").strip()
            print("\n")
            if choice == "1":
                check_in(username)
            elif choice == "2":
                check_out(username)
            elif choice == "3":
                print("Inventory:")
                show_inventory()
            elif choice == "4":
                print("Goodbye!")
                break
            else:
                print("Invalid choice.")

if __name__ == "__main__":
    main()


Welcome to the Inventory Management System
1. Login
2. Register
Enter your choice: 1


Enter username: melyna
Enter password: 123
Login successful!

Options:
1. Check in an object
2. Check out an object
3. View inventory
4. Exit
Enter your choice: 3


Inventory:
+-----+------+---------------+--------+--------------------+------------------+---------------------+-------------------+
| Bin | Slot | Serial Number | Status | Last Check In Date | Last Check In By | Last Check Out Date | Last Check Out By |
+-----+------+---------------+--------+--------------------+------------------+---------------------+-------------------+
+-----+------+---------------+--------+--------------------+------------------+---------------------+-------------------+

Options:
1. Check in an object
2. Check out an object
3. View inventory
4. Exit
Enter your choice: 4


Goodbye!
